In [3]:
# Installer le package elmNNRcpp si nécessaire
if (!requireNamespace("elmNNRcpp", quietly = TRUE)) {
  install.packages("elmNNRcpp")
}


library(elmNNRcpp)
library(dplyr)
library(caret)
library(tidyverse)

# Downloading packages -------------------------------------------------------
- Downloading elmNNRcpp from CRAN ...           OK [509.3 Kb in 0.8s]
- Downloading KernelKnn from CRAN ...           OK [244.5 Kb in 0.81s]
- Downloading RcppArmadillo from CRAN ...       OK [1 Mb in 0.43s]
Successfully downloaded 3 packages in 3.5 seconds.

The following package(s) will be installed:
- elmNNRcpp     [1.0.4]
- KernelKnn     [1.1.5]
- RcppArmadillo [14.0.0-1]
These packages will be installed into "~/renv/library/linux-ubuntu-jammy/R-4.4/x86_64-pc-linux-gnu".

# Installing packages --------------------------------------------------------
- Installing RcppArmadillo ...                  OK [installed binary and cached in 1.1s]
- Installing KernelKnn ...                      OK [installed binary and cached in 0.7s]
- Installing elmNNRcpp ...                      OK [installed binary and cached in 0.74s]


Loading required package: KernelKnn


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: ggplot2

Loading required package: lattice

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
✔ readr     2.1.5     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ purrr::lift()   masks caret::lift()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [1]:
stations_data <- read.csv(file = "stations_clim_data.csv") 
head(stations_data)

,Date,stations,tx,tn,rh,rs,ws,et0
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1988-01-01,bobo,29.5,15.0,32,15.70637,1.72,4.461520
2,1988-01-02,bobo,31.4,17.5,27,15.64143,1.67,4.731423
3,1988-01-03,bobo,27.5,17.8,22,13.36923,2.11,4.928952
4,1988-01-04,bobo,27.5,15.0,22,15.92401,1.89,4.762524
5,1988-01-05,bobo,29.3,14.4,24,15.28877,1.11,3.817034
6,1988-01-06,bobo,29.5,16.5,26,14.89933,0.78,3.357057


In [4]:
# Updated R code
stations_data <- read.csv(file = "stations_clim_data.csv") %>%
  mutate(across(tx:ws, ~scale(.x)))
head(stations_data)

,Date,stations,tx,tn,rh,rs,ws,et0
,<chr>,<chr>,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>",<dbl>
1,1988-01-01,bobo,-1.529612,-2.015924,-0.8135439,-0.5969087,1.1468711,4.461520
2,1988-01-02,bobo,-1.024084,-1.329384,-1.0362345,-0.6156664,1.0616517,4.731423
3,1988-01-03,bobo,-2.061747,-1.246999,-1.2589250,-1.2718984,1.8115830,4.928952
4,1988-01-04,bobo,-2.061747,-2.015924,-1.2589250,-0.5340548,1.4366173,4.762524
5,1988-01-05,bobo,-1.582826,-2.180694,-1.1698488,-0.7175171,0.1071937,3.817034
6,1988-01-06,bobo,-1.529612,-1.604000,-1.0807726,-0.8299899,-0.4552548,3.357057


In [3]:
predictors <- colnames(stations_data)[-c(1,2,8)]
predictors

[1] "tx" "tn" "rh" "rs" "ws"

In [4]:
stations_names=unique(stations_data$stations)
stations_names

[1] "bobo"       "boromo"     "dori"       "dedougou"   "fada"      
[6] "gaoua"      "ouaga"      "ouahigouya" "po"

In [20]:
# Entraîner le modèle ELM
set.seed(123)
elm_model <- elm_train(xtr,ytr, nhid = 10, actfun = "relu")
rr=elmNNRcpp::elm_predict(elm_model, xtr)

In [23]:
dim(rr)

[1] 10958     1

In [5]:
# Définir la fonction d'entraînement pour caret
train_elm <- function(xtr,ytr nhid, actfun) {
  elmNNRcpp::elm_train(xtr,ytr, nhid = nhid, actfun = actfun)
}

In [24]:
# Charger la bibliothèque caret
library(caret)

# Définir les fonctions pour l'entraînement et la prédiction
train_elm <- function(xtr,ytr, wts, param, lev, last, classProbs, ...) {
  model <- elmNNRcpp::elm_train(xtr,ytr, nhid = param$nhid, actfun = param$actfun)
  model
}

predict_elm <- function(modelFit, newdata, preProc = NULL, submodels = NULL) {
  elmNNRcpp::elm_predict(modelFit, newdata)
}

# Enregistrer la méthode de modélisation dans caret
model_info <- list(
  label = "Extreme Learning Machine",
  library = "elmNNRcpp",
  type = "Regression",
  parameters = data.frame(parameter = c("nhid", "actfun"), class = c("numeric", "character"), label = c("Number of Hidden Units", "Activation Function")),
  grid = function(x, y, len = NULL, search = "grid") expand.grid(nhid = seq(1, len, by = 1), actfun = c("sigmoid", "tanh")),
  fit = train_elm,
  predict = predict_elm,
  prob = NULL,
  tags = c("Neural Network", "Regression"),
  sort = function(x) x[order(x[, 1]),],
  levels = function(x) x$lev
)

# Ajouter le modèle à caret


In [25]:
# Configurer la validation croisée
train_control <- trainControl(method = "cv", number = 5)

In [ ]:
# Boucle sur chaque station
i=0
ns=length(stations_names)
for (station in stations_names) {
  i=i+1
  cat("Processing station:", station, " ( ", i,"/",ns," ) ","\n")
  
  # Filtrer les données pour la station courante
  station_data <- stations_data %>%
    filter(stations == station) %>%
    dplyr::select(et0, tx, tn, rh, rs, ws)
  
  # Division des données en ensembles d'entraînement et de test
  set.seed(123)
  train_index <- sample(1:nrow(station_data), 0.8 * nrow(station_data))
  train_data <- station_data[train_index, ]  # Corrected from mtcars to station_data
  test_data <- station_data[-train_index, ]  # Corrected from mtcars to station_data
  
  # Définir les hyperparamètres à tester
  grid <- expand.grid(
    nhid = seq(10, 50, 5),
    actfun = c("relu", "sigmoid", "tanh")
  )
  
  # Entraîner le modèle avec la recherche sur grille
  set.seed(123)
  elm_caret <- train(
    xtr,ytr,
    method = model_info,  
    trControl = train_control, 
    tuneGrid = grid
  )
  
  # Faire des prédictions sur les données de test
  test_predictions <- elmNNRcpp::elm_predict(elm_caret, newdata = test_data)
  
  # Calculer les métriques de performance
  rmse <- Metrics::rmse(actual = test_data$et0, predicted = test_predictions)  # Corrected from train_data to test_data
  
  cat("RMSE: ", rmse, "\n")
  
  # Sauvegarder le modèle final dans un fichier RDS
  saveRDS(elm_caret, file = paste0("elm_model_", station, ".rds"))
}

Processing station: bobo  (  1 / 9  )  


ERROR: Error: wrong model type for classification


In [ ]:
# Définir la fonction de modèle personnalisée pour ELM
train_elm <- function(xtr,ytr nhid, actfun) {
  elm_train(xtr,ytr, nhid = nhid, actfun = actfun)
}

predict_elm <- function(model, new_data) {
  elmNNRcpp::elm_predict(model, new_data)
}

# Définir la spécification du modèle ELM
elm_model_spec <- function(nhid, actfun) {
  new_model_spec <- list(
    nhid = nhid,
    actfun = actfun
  )
  class(new_model_spec) <- "elm_model_spec"
  new_model_spec
}

# Définir la méthode fit pour tidymodels
fit.elm_model_spec <- function(object, xtr,ytr, ...) {
  train_elm(xtr,ytr, object$nhid, object$actfun)
}

# Définir la méthode predict pour tidymodels
predict.elm_model_spec <- function(object, new_data, ...) {
  predict_elm(object, new_data)
}


In [ ]:
# Fonction pour évaluer chaque combinaison de la grille
evaluate_elm_model <- function(nhid, actfun) {
  elm_model <- elm_model_spec(nhid = nhid, actfun = actfun)
  resamples <- map(cv_folds$splits, ~ {
    train_split <- analysis(.x)
    test_split <- assessment(.x)
    fitted_model <- fit(elm_model, mpg ~ ., data = train_split)
    predictions <- predict(fitted_model, new_data = test_split)
    metrics <- metric_set(rmse, rsq)(truth = test_split$mpg, estimate = predictions)
    metrics
  })
  mean_metrics <- map_dfc(resamples, ~ summarise(.x, across(everything(), mean, na.rm = TRUE)))
  mean_metrics <- mean_metrics %>% mutate(nhid = nhid, actfun = actfun)
  mean_metrics
}

# Exécuter la recherche sur grille
results <- pmap_dfr(elm_grid, evaluate_elm_model)

# Sélectionner le meilleur modèle
best_model <- results %>% arrange(rmse) %>% slice(1)


ERROR: Error in eval(expr, envir, enclos): object 'elm_grid' not found


In [55]:
train_elm_model <- function(data, station_name) {
  # Filtrer les données pour la station spécifique
  station_data <- data %>% filter(stations == station_name) %>% select(-Date, -stations)
  
  # Diviser les données en ensembles d'entraînement et de test
  set.seed(123)
  data_split <- initial_split(station_data, prop = 0.8)
  train_data <- training(data_split)
  test_data <- testing(data_split)
  
  # Préparer les matrices pour elmNNRcpp
  xtr <- as.matrix(train_data %>% select(-et0))
  ytr <- as.matrix(train_data$et0)
  xte <- as.matrix(test_data %>% select(-et0))
  yte <- as.matrix(test_data$et0)
  
  # Définir les hyperparamètres pour la recherche sur grille
  nhid_values <- seq(5,50,5)
  actfun_values <- c("relu", "sig","tansig","radbas","hardlim","hardlims","satlins","tribas","purelin")
  
  # Initialiser les meilleures métriques
  best_rmse <- Inf
  best_model <- NULL
  best_params <- list()
  
  # Configuration de la validation croisée
  cv_splits <- vfold_cv(train_data, v = 5)
  
  # Recherche sur grille
  for (nhid in nhid_values) {
    for (actfun in actfun_values) {
      # Effectuer la validation croisée
      cv_results <- map(cv_splits$splits, function(split) {
        train_split <- analysis(split)
        test_split <- assessment(split)
        
        # Préparer les matrices pour elmNNRcpp
        xtr_cv <- as.matrix(train_split %>% select(-et0))
        ytr_cv <- as.matrix(train_split$et0)
        xte_cv <- as.matrix(test_split %>% select(-et0))
        yte_cv <- as.matrix(test_split$et0)
        
        # Entraîner le modèle ELM
        fit_elm <- elm_train(xtr_cv, ytr_cv, nhid = nhid, actfun = actfun,
                             init_weights = "uniform_negative", bias = TRUE, verbose = FALSE)
        
        # Faire des prédictions sur l'ensemble de test
        predictions <- elm_predict(fit_elm, xte_cv)
        
        # Calculer le RMSE
        rmse_value <- sqrt(mean((yte_cv - predictions)^2))
        return(rmse_value)
      })
      
      # Calculer le RMSE moyen pour cette combinaison d'hyperparamètres
      mean_rmse <- mean(unlist(cv_results))
      
      # Si le RMSE est meilleur, sauvegarder le modèle et les paramètres
      if (mean_rmse < best_rmse) {
        best_rmse <- mean_rmse
        best_model <- elm_train(xtr, ytr, nhid = nhid, actfun = actfun,
                                init_weights = "uniform_negative", bias = TRUE, verbose = FALSE)
        best_params <- list(nhid = nhid, actfun = actfun)
      }
    }
  }
  
  # Évaluer les performances du meilleur modèle
  final_predictions <- elm_predict(best_model, xte)
  final_rmse <- sqrt(mean((yte - final_predictions)^2))
  final_rsq <- cor(yte, final_predictions)^2
  
  cat("Station:", station_name, "\n")
  cat("Best Parameters: nhid =", best_params$nhid, ", actfun =", best_params$actfun, "\n")
  cat("RMSE:", final_rmse, "\n")
  cat("R²:", final_rsq, "\n\n")
  
  return(list(model = best_model, rmse = final_rmse, rsq = final_rsq, params = best_params))
}

In [46]:
# Lancer l'entraînement pour chaque station
stations <- unique(stations_data$stations)
stations

[1] "bobo"       "boromo"     "dori"       "dedougou"   "fada"      
[6] "gaoua"      "ouaga"      "ouahigouya" "po"

In [40]:
library(tidymodels)
library(elmNNRcpp)
library(dplyr)
library(purrr)

── Attaching packages ────────────────────────────────────── tidymodels 1.2.0 ──

✔ broom        1.0.6      ✔ rsample      1.2.1 
✔ dials        1.2.1      ✔ tune         1.2.1 
✔ infer        1.0.7      ✔ workflows    1.1.4 
✔ modeldata    1.3.0      ✔ workflowsets 1.1.0 
✔ parsnip      1.2.1      ✔ yardstick    1.3.1 
✔ recipes      1.0.10     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard()        masks purrr::discard()
✖ dplyr::filter()          masks stats::filter()
✖ recipes::fixed()         masks stringr::fixed()
✖ dplyr::lag()             masks stats::lag()
✖ purrr::lift()            masks caret::lift()
✖ yardstick::precision()   masks caret::precision()
✖ yardstick::recall()      masks caret::recall()
✖ yardstick::sensitivity() masks caret::sensitivity()
✖ yardstick::spec()        masks readr::spec()
✖ yardstick::specificity() masks caret::specificity()
✖ recipes::step()          masks stats::step()
• Use tidymodels_prefer(

In [56]:
library(purrr)
library(dplyr)

# Assuming 'stations' is a vector of station names and 'data' is a dataframe
results <- purrr::map(stations, ~train_elm_model(stations_data, .x))


Station: bobo 
Best Parameters: nhid = 50 , actfun = sig 
RMSE: 0.08449702 
R²: 0.9940615 

Station: boromo 
Best Parameters: nhid = 50 , actfun = sig 
RMSE: 0.08026991 
R²: 0.9926382 

Station: dori 
Best Parameters: nhid = 50 , actfun = sig 
RMSE: 0.09993766 
R²: 0.9904981 

Station: dedougou 
Best Parameters: nhid = 45 , actfun = sig 
RMSE: 0.1093115 
R²: 0.9916518 

Station: fada 
Best Parameters: nhid = 45 , actfun = sig 
RMSE: 0.09348472 
R²: 0.9912127 

Station: gaoua 
Best Parameters: nhid = 45 , actfun = sig 
RMSE: 0.07724119 
R²: 0.9933984 

Station: ouaga 
Best Parameters: nhid = 45 , actfun = sig 
RMSE: 0.09992828 
R²: 0.9916435 

Station: ouahigouya 
Best Parameters: nhid = 45 , actfun = sig 
RMSE: 0.1210696 
R²: 0.9888801 

Station: po 
Best Parameters: nhid = 50 , actfun = sig 
RMSE: 0.08410406 
R²: 0.9925222 

